## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,dourbali,11.8091,15.8632,70.86,19,24,10.49,TD,2021-01-23 22:44:14,few clouds
1,1,masalli,34.7959,126.6553,39.20,100,40,2.30,KR,2021-01-23 22:44:14,fog
2,2,puerto ayora,-0.7393,-90.3518,80.60,74,40,9.22,EC,2021-01-23 22:40:11,scattered clouds
3,3,srednekolymsk,67.4500,153.6833,-28.32,84,100,9.22,RU,2021-01-23 22:42:16,overcast clouds
4,4,tiksi,71.6872,128.8694,-41.78,79,35,5.03,RU,2021-01-23 22:41:33,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
2,2,puerto ayora,-0.7393,-90.3518,80.60,74,40,9.22,EC,2021-01-23 22:40:11,scattered clouds
5,5,vaini,-21.2000,-175.2000,82.40,83,75,5.75,TO,2021-01-23 22:41:17,light rain
17,17,caravelas,-17.7125,-39.2481,75.31,82,47,4.94,BR,2021-01-23 22:41:20,scattered clouds
21,21,jizan,17.3333,42.6667,75.20,78,0,2.01,SA,2021-01-23 22:41:28,clear sky
22,22,werda,-25.2667,23.2833,80.69,19,62,5.30,BW,2021-01-23 22:44:17,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                1
Date                   0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,puerto ayora,EC,80.60,scattered clouds,-0.7393,-90.3518,
5,vaini,TO,82.40,light rain,-21.2000,-175.2000,
17,caravelas,BR,75.31,scattered clouds,-17.7125,-39.2481,
21,jizan,SA,75.20,clear sky,17.3333,42.6667,
22,werda,BW,80.69,broken clouds,-25.2667,23.2833,
23,port blair,IN,75.31,scattered clouds,11.6667,92.7500,
25,atuona,PF,77.99,light rain,-9.8000,-139.0333,
27,nova olinda do norte,BR,78.75,heavy intensity rain,-3.8881,-59.0942,
36,rikitea,PF,79.21,clear sky,-23.1203,-134.9692,
37,kapaa,US,75.20,light rain,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == ""].index)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,puerto ayora,EC,80.60,scattered clouds,-0.7393,-90.3518,Castillo Galapagos
5,vaini,TO,82.40,light rain,-21.2000,-175.2000,Keleti Beach Resort
17,caravelas,BR,75.31,scattered clouds,-17.7125,-39.2481,Pousada dos Navegantes
21,jizan,SA,75.20,clear sky,17.3333,42.6667,شهدان. ابوالحجر
23,port blair,IN,75.31,scattered clouds,11.6667,92.7500,Welcomhotel Bay Island Port Blair
...,...,...,...,...,...,...,...
681,kailua,US,80.60,overcast clouds,21.4022,-157.7394,Sheffield House Bed & Breakfast
685,angoche,MZ,79.27,overcast clouds,-16.2325,39.9086,Guest House HESADA
691,manicore,BR,79.66,scattered clouds,-5.8092,-61.3003,Hotel Durval
693,poum,NC,81.05,overcast clouds,-20.2333,164.0167,Gîte kajeon


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))